# Optimización de Pesos de Evaluación

Este notebook tiene como objetivo encontrar la combinación óptima de pesos para los items de evaluación definidos en `config_meccel.py`.

El objetivo es minimizar la discrepancia entre las notas calculadas por el modelo matemático (Método Avanzado) y las decisiones heurísticas de un "Oráculo" (reglas de negocio experto).

Se busca minimizar:
$$ Costo = FPR + FNR $$
Donde:
- **FPR (False Positive Rate)**: Estudiantes que el modelo aprueba pero el Oráculo reprueba.
- **FNR (False Negative Rate)**: Estudiantes que el modelo reprueba pero el Oráculo aprueba.

In [1]:
import pandas as pd
import numpy as np
import copy
from scipy.optimize import minimize

# Módulos propios
import notas as nu

pd.set_option('display.float_format', '{:.4f}'.format)
np.random.seed(42)

## 1. Configuración Inicial y Generación de Datos

In [2]:
# import config_meccel as cf
import config_astrobio as cf

# Cargar Configuración
config_inicial = nu.autoconfigura_items(cf.config_evaluacion)

# Generar Datos Sintéticos (N grande para estabilidad estadística)
df = nu.genera_datos(config_inicial, N=2000)

# Pre-calcular la decisión del Oráculo (Ground Truth)
df['Decision_Oraculo'] = df.apply(lambda row: nu.calcula_decision_oraculo(row, config_inicial), axis=1)

print(f"Datos generados: {len(df)} estudiantes")
print("Distribución del Oráculo:")
print(df['Decision_Oraculo'].value_counts(normalize=True))

nu.muestra_pesos(config_inicial)

Autoconfigurando items correlacionados...
 > Quices_Mod2: Ajustado vs Quices_Mod1 (x1.2)
 > Tarea_Mod2: Ajustado vs Tarea_Mod1 (x1.2)
 > Quices_Mod3: Ajustado vs Quices_Mod1 (x1.0)
 > Tarea_Mod3: Ajustado vs Tarea_Mod1 (x1.0)
Datos generados: 2000 estudiantes
Distribución del Oráculo:
Decision_Oraculo
0   0.5370
1   0.4630
Name: proportion, dtype: float64

PESOS FINALES DE EVALUACIÓN
ITEM                      PESO      
-----------------------------------
Quices_Mod1               0.1500
Tarea_Mod1                0.1500
Quices_Mod2               0.1800
Tarea_Mod2                0.1800
Quices_Mod3               0.1500
Tarea_Mod3                0.1500
Examen Final              0.0400
-----------------------------------
TOTAL SUMA                1.0000


In [3]:
nu.mostrar_notas(df, config_inicial, 10)

,"Quices_Mod1 [15.0%, avanzado]","Tarea_Mod1 [15.0%, avanzado]","Quices_Mod2 [18.0%, facil]","Tarea_Mod2 [18.0%, clave]","Quices_Mod3 [15.0%, facil]","Tarea_Mod3 [15.0%, avanzado]","Examen Final [4.0%, examen]",Decision_Oraculo
1860,2.7700,2.0300,3.3900,0.7400,4.4900,4.5800,3.8000,1
353,2.7100,3.0300,2.9000,1.3600,2.5800,3.0800,1.5300,0
1333,1.1300,3.8600,3.8200,3.8400,4.4300,2.4500,3.2300,0
905,2.9400,1.4000,4.1700,1.2700,3.1100,3.8100,2.5700,0
1289,3.1000,3.3300,4.1800,5.0000,3.9900,3.0900,4.4900,1
1273,2.2600,1.2300,4.0700,0.0400,3.3500,4.6700,2.9400,0
938,2.3200,2.6400,2.6900,3.9500,4.3400,1.8100,2.4200,0
1731,3.0800,2.7500,4.3400,0.5500,2.5300,2.2300,3.8300,1
65,2.1300,2.5400,4.0800,1.3700,4.0800,3.4800,4.5600,1
1323,2.3600,2.7600,4.0100,0.0300,3.4800,0.7300,2.7500,0


## 2. Definición de la Función Objetivo

Definimos la función que el optimizador minimizará. Esta función toma los pesos independientes, recalcula las notas y compara con el Oráculo.

In [4]:
PROMEDIO_OBJETIVO = "Promedio_Clasico" 
# Otros son: Nota_Final, 

def funcion_objetivo(x_pesos, config_base, df_data, mapping_info):
    # 1. Actualizar configuración
    config_temp = copy.deepcopy(config_base)
    config_temp = nu.actualizar_pesos_desde_vector(x_pesos, config_temp, mapping_info)
    
    # 2. Verificar Restricciones del Item Definitorio (PENALIZACIÓN)
    # Si el peso del examen final excede el máximo permitido, penalizamos el costo.
    def_item = config_temp['item_definitorio']
    w_def = def_item.get('peso_final', 0.0)
    limit_max = def_item.get('peso_maximo', 1.0)
    
    penalty = 0.0
    if w_def > limit_max:
         diff = w_def - limit_max
         # Penalización fuerte para obligar al optimizador a "subir" los otros pesos
         penalty = diff * 100.0 
    
    # 3. Calcular Notas (Método Avanzado)
    df_res = nu.calcula_promedio_con_umbrales_avanzado(df_data, config_temp)
    
    # 4. Métricas
    metrics = nu.analisis_falsos_positivos_negativos(df_res, PROMEDIO_OBJETIVO, config_temp)
    
    return metrics['FPR'] + metrics['FNR'] + penalty

## 3. Configuración del Optimizador

Identificamos los items independientes para optimizar solo sus dimensiones.

In [5]:
# Preparar variables de optimización automáticamente
# Esto incluye items normales y nota_concepto si aplica
x0, bounds, mapping_info = nu.preparar_variables_optimizacion(config_inicial)

print("Variables de optimización:")
for i, info in enumerate(mapping_info):
    print(f"  x[{i}]: {info['name']} ({info['type']})")

print(f"\nValores iniciales: {x0}")
print(f"Límites: {bounds}")

Variables de optimización:
  x[0]: Quices_Mod1 (normal)
  x[1]: Tarea_Mod1 (normal)

Valores iniciales: [0.15 0.15]
Límites: [(0.05, 0.2), (0.05, 0.2)]


## 4. Ejecutar Optimización

Utilizamos `scipy.optimize.minimize` con el método SLSQP que permite restricciones de caja (límites).

In [6]:
print("Iniciando optimización (Método Powell con Penalización)...")

# Usamos el método 'Powell' o 'Nelder-Mead' porque la función objetivo (FPR+FNR)
# es discontinua (tipo escalón) y los métodos basados en gradiente (como SLSQP) fallan
# al ver un gradiente de 0.
resultado = minimize(
    fun=funcion_objetivo,
    x0=x0,
    args=(config_inicial, df, mapping_info), # Pass mapping_info inside args
    method='Powell', # Changed from SLSQP
    bounds=bounds,
    tol=1e-4
)

print("\nResultado de la optimización:")
print(resultado)

Iniciando optimización (Método Powell con Penalización)...

Resultado de la optimización:
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.4937517847734192
       x: [ 7.343e-02  1.457e-01]
     nit: 2
   direc: [[ 1.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00]]
    nfev: 61


## 5. Análisis de Resultados

Comparamos la configuración inicial con la optimizada.

In [7]:
def evaluar_configuracion(pesos_vec, nombre_caso):
    costo = funcion_objetivo(pesos_vec, config_inicial, df, mapping_info)
    
    # Recalcular para mostrar detalles
    config_temp = copy.deepcopy(config_inicial)
    config_temp = nu.actualizar_pesos_desde_vector(pesos_vec, config_temp, mapping_info)
    
    df_res = nu.calcula_promedio_con_umbrales_avanzado(df, config_temp)
    metrics = nu.analisis_falsos_positivos_negativos(df_res, PROMEDIO_OBJETIVO, config_temp)
    
    print(f"\n--- {nombre_caso} ---")
    print(f"Costo (FPR+FNR): {costo:.4f}")
    print(f"FPR: {metrics['FPR']:.2%}")
    print(f"FNR: {metrics['FNR']:.2%}")
    print(f"Accuracy: {metrics['Accuracy']:.2%}")
    
    nu.muestra_pesos(config_temp)

print("COMPARACIÓN PRE VS POST OPTIMIZACIÓN")
evaluar_configuracion(x0, "Configuración Inicial (Sugerida)")
evaluar_configuracion(resultado.x, "Configuración Optimizada")

COMPARACIÓN PRE VS POST OPTIMIZACIÓN

--- Configuración Inicial (Sugerida) ---
Costo (FPR+FNR): 0.7968
FPR: 59.59%
FNR: 20.09%
Accuracy: 58.70%

PESOS FINALES DE EVALUACIÓN
ITEM                      PESO      
-----------------------------------
Quices_Mod1               0.1500
Tarea_Mod1                0.1500
Quices_Mod2               0.1800
Tarea_Mod2                0.1800
Quices_Mod3               0.1500
Tarea_Mod3                0.1500
Examen Final              0.0400
-----------------------------------
TOTAL SUMA                1.0000

--- Configuración Optimizada ---
Costo (FPR+FNR): 0.4938
FPR: 30.26%
FNR: 19.11%
Accuracy: 74.90%

PESOS FINALES DE EVALUACIÓN
ITEM                      PESO      
-----------------------------------
Quices_Mod1               0.0734
Tarea_Mod1                0.1457
Quices_Mod2               0.0881
Tarea_Mod2                0.1748
Quices_Mod3               0.0734
Tarea_Mod3                0.1457
Examen Final              0.2989
----------------------

In [8]:
print("\n=== ANÁLISIS DETALLADO DE CASOS DE ERROR ===")

# 1. Recalcular todo con Configuración Inicial para identificar índices
config_temp_init = copy.deepcopy(config_inicial)
config_temp_init = nu.actualizar_pesos_desde_vector(x0, config_temp_init, mapping_info)
df_init = nu.calcula_promedio_con_umbrales_avanzado(df, config_temp_init)

# Definir condiciones de error (Threshold modelo = 2.95 según función avanzada)
# FPR: Oraculo = 0 (Reprueba), Modelo >= 2.95 (Aprueba)
# FNR: Oraculo = 1 (Aprueba), Modelo < 2.95 (Reprueba)
umbral_aprobacion = 2.95
mask_fpr = (df_init['Decision_Oraculo'] == 0) & (df_init['Nota_Final'] >= umbral_aprobacion)
mask_fnr = (df_init['Decision_Oraculo'] == 1) & (df_init['Nota_Final'] < umbral_aprobacion)

idx_fpr = df_init[mask_fpr].index[:5].tolist() if any(mask_fpr) else []
idx_fnr = df_init[mask_fnr].index[:5].tolist() if any(mask_fnr) else []

print(f"\n--- Ejemplos de Falsos Positivos Iniciales (Total: {mask_fpr.sum()}) ---")
if len(idx_fpr) > 0:
    # Usamos display() si estamos en un notebook (Jupyter lo inyecta), sino print
    try:
        display(nu.mostrar_notas(df_init.loc[idx_fpr], config_temp_init))
    except NameError:
        print(nu.mostrar_notas(df_init.loc[idx_fpr], config_temp_init))
else:
    print("No se encontraron Falsos Positivos con la configuración inicial.")

print(f"\n--- Ejemplos de Falsos Negativos Iniciales (Total: {mask_fnr.sum()}) ---")
if len(idx_fnr) > 0:
    try:
        display(nu.mostrar_notas(df_init.loc[idx_fnr], config_temp_init))
    except NameError:
        print(nu.mostrar_notas(df_init.loc[idx_fnr], config_temp_init))
else:
    print("No se encontraron Falsos Negativos con la configuración inicial.")


# 2. Mostrar los MISMOS estudiantes con la Configuración Optimizada
print("\n\n=== IMPACTO DE LA OPTIMIZACIÓN EN ESTOS CASOS ===")
config_temp_opt = copy.deepcopy(config_inicial)
config_temp_opt = nu.actualizar_pesos_desde_vector(resultado.x, config_temp_opt, mapping_info)
df_opt = nu.calcula_promedio_con_umbrales_avanzado(df, config_temp_opt)

# Asegurar que tenemos los mismos estudiantes calculados
indices_interes = idx_fpr + idx_fnr
df_view_opt = df_opt.loc[indices_interes]

if len(df_view_opt) > 0:
    try:
        display(nu.mostrar_notas(df_view_opt, config_temp_opt))
    except NameError:
        print(nu.mostrar_notas(df_view_opt, config_temp_opt))
else:
    print("No hay casos para mostrar.")


=== ANÁLISIS DETALLADO DE CASOS DE ERROR ===

--- Ejemplos de Falsos Positivos Iniciales (Total: 226) ---


,"Quices_Mod1 [15.0%, avanzado]","Tarea_Mod1 [15.0%, avanzado]","Quices_Mod2 [18.0%, facil]","Tarea_Mod2 [18.0%, clave]","Quices_Mod3 [15.0%, facil]","Tarea_Mod3 [15.0%, avanzado]","Examen Final [4.0%, examen]",Promedio_Clasico,Nota_Final,Decision_Oraculo
8,4.3600,2.4200,4.4300,4.6800,3.8900,2.9300,1.3500,3.7300,3.0000,0
30,3.0600,2.5600,4.2200,2.1400,3.4900,2.3200,1.4300,3.0000,3.0000,0
36,2.7500,2.5800,4.2900,3.1400,4.2400,2.3800,2.2400,3.2200,3.0000,0
82,2.6000,2.9400,3.5000,4.0200,4.4500,2.7400,2.9000,3.3800,3.3100,0
103,2.8300,2.9200,3.4700,3.9100,2.5600,3.3700,2.4800,3.1800,3.1600,0



--- Ejemplos de Falsos Negativos Iniciales (Total: 589) ---


,"Quices_Mod1 [15.0%, avanzado]","Tarea_Mod1 [15.0%, avanzado]","Quices_Mod2 [18.0%, facil]","Tarea_Mod2 [18.0%, clave]","Quices_Mod3 [15.0%, facil]","Tarea_Mod3 [15.0%, avanzado]","Examen Final [4.0%, examen]",Promedio_Clasico,Nota_Final,Decision_Oraculo
3,3.0500,3.7500,4.3700,0.4800,4.4900,3.5100,4.0400,3.2500,1.0200,1
5,2.6800,3.7200,4.4400,1.6700,3.7600,3.9100,4.0200,3.3700,2.7500,1
6,3.2400,1.3600,3.2000,0.8500,4.2800,3.4900,2.9600,2.7000,2.7000,1
9,3.4900,3.2600,4.2400,1.0800,4.4000,4.3000,4.1600,3.4400,2.0700,1
14,5.0000,4.1200,4.1600,0.8500,4.2900,2.2100,3.1400,3.3700,1.4900,1




=== IMPACTO DE LA OPTIMIZACIÓN EN ESTOS CASOS ===


,"Quices_Mod1 [7.3%, avanzado]","Tarea_Mod1 [14.6%, avanzado]","Quices_Mod2 [8.8%, facil]","Tarea_Mod2 [17.5%, clave]","Quices_Mod3 [7.3%, facil]","Tarea_Mod3 [14.6%, avanzado]","Examen Final [29.9%, examen]",Promedio_Clasico,Nota_Final,Decision_Oraculo
8,4.3600,2.4200,4.4300,4.6800,3.8900,2.9300,1.3500,3.0000,2.7000,0
30,3.0600,2.5600,4.2200,2.1400,3.4900,2.3200,1.4300,2.3700,2.3700,0
36,2.7500,2.5800,4.2900,3.1400,4.2400,2.3800,2.2400,2.8300,2.8300,0
82,2.6000,2.9400,3.5000,4.0200,4.4500,2.7400,2.9000,3.2200,3.1700,0
103,2.8300,2.9200,3.4700,3.9100,2.5600,3.3700,2.4800,3.0400,3.0300,0
3,3.0500,3.7500,4.3700,0.4800,4.4900,3.5100,4.0400,3.2900,1.0200,1
5,2.6800,3.7200,4.4400,1.6700,3.7600,3.9100,4.0200,3.4700,2.8000,1
6,3.2400,1.3600,3.2000,0.8500,4.2800,3.4900,2.9600,2.5700,2.5700,1
9,3.4900,3.2600,4.2400,1.0800,4.4000,4.3000,4.1600,3.4900,2.0900,1
14,5.0000,4.1200,4.1600,0.8500,4.2900,2.2100,3.1400,3.0600,1.4200,1
